In [1]:
import torch
from LIM.neural_networks import LIM_class
from neural_networks.utilities import *

%load_ext autoreload
%autoreload 2

# Create the DataLoader for first principal component
data = torch.load("./neural_networks/data/data_piControl.pt")
print("Data shape : {}".format(data.shape))

# Reshape the data if necessary (assuming a 2D tensor)
if len(data.shape) == 1:
    data = data.unsqueeze(1)

data = normalize_data(data)

Data shape : torch.Size([30, 14400])


In [2]:
eps = 0.001

# original fit
tau = 1
model_org = LIM_class.LIM(tau)
model_org.fit(data.detach().cpu().numpy(), eps=eps)

print("Data [:, -1] : {}".format(data[:, -1]))
lim_integration, times_ = model_org.noise_integration(data[:, -1], timesteps=100000, num_comp=30)
lim_integration = lim_integration.T

lim_integration = torch.from_numpy(lim_integration)
torch.save(lim_integration, "./neural_networks/synthetic_data/lim_integration_100k[-1].pt")
print("Lim integration shape : {}".format(lim_integration.shape))
print("Saved data")

Covariance matrix has negative values!
Data [:, -1] : tensor([-1.0789,  0.6009,  2.1929,  1.9903, -0.0526, -0.2151,  1.5943, -0.0291,
         0.4448, -0.8276,  0.8989,  0.9157, -2.0359, -0.2615, -0.5470, -2.0026,
         0.2789, -0.8578, -0.4140, -0.8562, -1.7976,  1.7037, -1.4693,  1.4704,
         1.0043, -0.2691, -0.3022, -0.8519,  0.0362,  0.0679])
state_start: 30 and type : <class 'numpy.ndarray'>
t_decay: 2.178929869852326
t_delta: 1
Lim integration shape : torch.Size([30, 100001])
Saved data
